In [26]:
import os
import numpy as np
import pandas as pd
import re
# Define the folder path where your text files are located
folder_path = '/Users/safdarabbas/Desktop/Safdar /deep learning/safdar/ss'
# Initialize an empty list to hold the stories
stories = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):  # Only process .txt files
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            story_text = file.read()
            stories.append(story_text)

# Check how many stories were loaded
print(f"Loaded {len(stories)} stories.")


Loaded 2 stories.


In [37]:
def clean_text(doc):
    # Remove the title and author section (generalized for different titles)
    doc = re.sub(r'^.*\n\s*.*\n+', '', doc, flags=re.IGNORECASE)

    # Remove everything below the dashed line
    doc = re.sub(r'\n[-]{10,}.*$', '', doc, flags=re.DOTALL)

    # Remove special characters and digits
    doc = re.sub(r'\W', ' ', str(doc))

    # Remove single characters (like 'a', 'b', etc.)
    doc = re.sub(r'\s+[a-zA-Z]\s+', ' ', doc)

    # Replace multiple spaces with one space
    doc = re.sub(r'\s+', ' ', doc)

    # Convert to lowercase
    doc = doc.lower()

    return doc
cleaned_txt=[]    
for story in stories:    
    cleaned_txt.append(clean_text(story))
print(cleaned_txt[0][:1000])    

the adventure of the three garridebs arthur conan doyle it may have been a comedy or it may have been a tragedy it cost one man his reason it cost me a blood letting and it cost yet another man the penalties of the law yet there was certainly an element of comedy well you shall judge for yourselves i remember the date very well for it was in the same month that holmes refused a knighthood for services which may perhaps some day be described i only refer to the matter in passing for in my position of partner and confidant i am obliged to be particularly careful to avoid any indiscretion i repeat however that this enables me to fix the date which was the latter end of june 1902 shortly after the conclusion of the south african war holmes had spent several days in bed as was his habit from time to time but he emerged that morning with a long foolscap document in his hand and a twinkle of amusement in his austere gray eyes there is a chance for you to make some money friend watson said he 

In [62]:
def get_summary_and_text(text, word_limit=100):
    words = text.split()
    summary = ' '.join(words[:word_limit])
    input_text = ' '.join(words[0:])
    return input_text, summary

input_texts = []
summaries = []

for story in stories:
    input_text, summary = get_summary_and_text(story)
    input_texts.append(input_text)
    summaries.append(summary)

print('INPUT TXT',input_texts[0][:1000])
print('summery',summaries[0])

INPUT TXT THE ADVENTURE OF THE THREE GARRIDEBS Arthur Conan Doyle It may have been a comedy, or it may have been a tragedy. It cost one man his reason, it cost me a blood-letting, and it cost yet another man the penalties of the law. Yet there was certainly an element of comedy. Well, you shall judge for yourselves. I remember the date very well, for it was in the same month that Holmes refused a knighthood for services which may perhaps some day be described. I only refer to the matter in passing, for in my position of partner and confidant I am obliged to be particularly careful to avoid any indiscretion. I repeat, however, that this enables me to fix the date, which was the latter end of June, 1902, shortly after the conclusion of the South African War. Holmes had spent several days in bed, as was his habit from time to time, but he emerged that morning with a long foolscap document in his hand and a twinkle of amusement in his austere gray eyes. "There is a chance for you to make s

In [81]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

tokenizer = Tokenizer()

tokenizer.fit_on_texts(input_texts + summaries)

start_token = "<start>"
end_token = "<end>"
if start_token not in tokenizer.word_index:
    tokenizer.word_index[start_token] = len(tokenizer.word_index) + 1
if end_token not in tokenizer.word_index:
    tokenizer.word_index[end_token] = len(tokenizer.word_index) + 2

tokenizer.word_index["<pad>"] = 0
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 for padding token

input_sequences = tokenizer.texts_to_sequences(input_texts)
summary_sequences = tokenizer.texts_to_sequences(summaries)

max_encoder_seq_length = max([len(seq) for seq in input_sequences])
max_decoder_seq_length = max([len(seq) for seq in summary_sequences])

encoder_input_seq = pad_sequences(input_sequences, maxlen=max_encoder_seq_length, padding="post")
decoder_input_seq = pad_sequences(summary_sequences, maxlen=max_decoder_seq_length, padding="post")

decoder_target_seq = np.zeros_like(decoder_input_seq)
decoder_target_seq[:, :-1] = decoder_input_seq[:, 1:]

# Encoder
encoder_inputs = Input(shape=(max_encoder_seq_length,))
encoder_embedding = Embedding(input_dim=vocab_size, output_dim=64)(encoder_inputs)
encoder_lstm = LSTM(64, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_decoder_seq_length,))
decoder_embedding = Embedding(input_dim=vocab_size, output_dim=64)(decoder_inputs)
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Fully connected layer for prediction
decoder_dense = Dense(vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

# Train the model
model.fit(
    [encoder_input_seq, decoder_input_seq], np.expand_dims(decoder_target_seq, axis=-1),
    batch_size=1,
    epochs=10
)

# Encoder inference model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder inference setup
decoder_state_input_h = Input(shape=(64,))
decoder_state_input_c = Input(shape=(64,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs_single = Input(shape=(1,))
decoder_embedding_inf = Embedding(input_dim=vocab_size, output_dim=64)(decoder_inputs_single)

decoder_lstm_inf = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm_inf(
    decoder_embedding_inf, initial_state=decoder_states_inputs
)
decoder_states_inf = [state_h_inf, state_c_inf]

decoder_outputs_inf = decoder_dense(decoder_outputs_inf)

decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs_inf] + decoder_states_inf
)

def decode_sequence(input_seq):
    # Encode the input sequence to get the context vectors
    states_value = encoder_model.predict(input_seq)

    # Initialize the target sequence with the start token
    target_seq = np.zeros((1, 1))
    
    # Ensure that the start token exists in the word index
    start_token_index = tokenizer.word_index.get(start_token, None)
    if start_token_index is None:
        raise ValueError(f"Start token '{start_token}' not found in tokenizer word index.")
    
    target_seq[0, 0] = start_token_index  # Replace with your actual start token index

    decoded_sentence = ""

    for _ in range(max_decoder_seq_length):
        # Predict the next word
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Get the index of the most likely word
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_token_index, "")

        # Append the word to the decoded sentence
        decoded_sentence += " " + sampled_word

        # Exit if the end token is predicted
        if sampled_word == end_token or sampled_word == "":
            break

        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence

input_seq = pad_sequences(
    tokenizer.texts_to_sequences(['''he door had flown open and a huge negro had burst into the room. He
     would have been a comic figure if he had not been terrific, for he
     was dressed in a very loud gray check suit with a flowing
     salmon-coloured tie. His broad face and flattened nose were thrust
     forward, as his sullen dark eyes, with a smouldering gleam of malice
     in them, turned from one of us to the other''']), maxlen=max_encoder_seq_length, padding="post"
)

# Decode the sequence
predicted_summary = decode_sequence(input_seq)
print("Predicted Summary:", predicted_summary)


Epoch 1/10


2025-01-06 04:37:35.559795: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-01-06 04:37:35.560177: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-01-06 04:37:35.560596: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2/2 [==============================] - 3s 1s/step - loss: 7.7688
Epoch 2/10
2/2 [==============================] - 2s 1s/step - loss: 7.7638
Epoch 3/10
2/2 [==============================] - 2s 1s/step - loss: 7.7584
Epoch 4/10
2/2 [==============================] - 2s 1s/step - loss: 7.7521
Epoch 5/10
2/2 [==============================] - 2s 1s/step - loss: 7.7436
Epoch 6/10
2/2 [==============================] - 2s 1s/step - loss: 7.7316
Epoch 7/10
2/2 [==============================] - 2s 1s/step - loss: 7.7120
Epoch 8/10
2/2 [==============================] - 2s 1s/step - loss: 7.6757
Epoch 9/10
2/2 [==============================] - 2s 1s/step - loss: 7.5907
Epoch 10/10
1/1 [==============================] - 0s 165ms/step


2025-01-06 04:37:57.300037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-01-06 04:37:57.300641: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-01-06 04:37:57.301009: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 6ms/step


2025-01-06 04:37:57.547476: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-01-06 04:37:57.547912: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-01-06 04:37:57.548320: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 7ms/step
Predicted Summary:  adventure adventure adventure adventure adventure adventure adventure adventure adventure adventure adventure adventure adventure adventure adventure me me me me me me well well well well or or or or or of of of of of 
